In [1]:
import os
os.chdir('..')

In [2]:
from src.data.data_cleaning import DataCleaning

In [3]:
data_cleaning = DataCleaning(main_path="data/raw")

In [4]:
leads_df = data_cleaning.load_dataset(dataset_path="leads.csv")
data_cleaning.inspect_dataset(leads_df)
data_cleaning.identify_columns_with_single_value(leads_df)
data_cleaning.identify_row_with_duplicate_data(leads_df)


Shape: (61639, 10)

         Id       First Name          Use Case   Source     Status  \
0  23okrabh              NaN  Corporate Events  Inbound  Nurturing   
1       NaN  Taylor1 Harris1               NaN  Inbound  Discarded   
2  hbzt0kp1   Alex6 Jackson3               NaN  Inbound  Nurturing   
3  velluwdq  Bailey2 Taylor6  Corporate Events  Inbound  Discarded   
4       NaN    Drew5 Wilson5  Corporate Events  Inbound  Nurturing   

  Discarded/Nurturing Reason    Acquisition Campaign Created Date  Converted  \
0                    Not Fit                     NaN   2018-05-26          0   
1                Not feeling  Event Management Guide   2018-10-17          0   
2                 Competitor                     NaN   2019-03-29          0   
3                 Not Target                     NaN   2018-05-26          0   
4                 Competitor                     NaN   2018-10-17          0   

      City  
0  Chicago  
1   Denver  
2   Denver  
3      NaN  
4      NaN  

In [5]:
offers_df = data_cleaning.load_dataset(dataset_path="offers.csv")
data_cleaning.inspect_dataset(offers_df)
data_cleaning.identify_columns_with_single_value(offers_df)
data_cleaning.identify_row_with_duplicate_data(offers_df)


Shape: (6130, 9)

         Id              Use Case       Status Created Date  Close Date  \
0  doqyhjtv  Educational Seminars   Closed Won   2019-11-17  2019-11-17   
1       NaN         Sports Events  Closed Lost   2019-11-24  2019-11-25   
2       NaN         Sports Events  Closed Lost   2019-11-24  2019-11-25   
3  cturqi48      Corporate Events   Closed Won   2019-05-07  2019-05-07   
4       NaN         Sports Events  Closed Lost   2019-11-17  2019-11-25   

   Price    Discount code               Pain  Loss Reason  
0  300.0       SAVEMORE50         operations          NaN  
1  150.0              NaN         operations        price  
2  768.0      HALFYEAR20G         operations  no response  
3  240.0  DISCOUNT50POWER  financial control          NaN  
4  240.0     SAVEMONTH50G  financial control  no response  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6130 entries, 0 to 6129
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------        

When analyzing the data we can observe:
* There are more records per leads than per offers.
* There are null values.
* There are no columns with a single value.
* There are rows that contain duplicate data.

In [6]:
leads_df = data_cleaning.delete_row_with_duplicate_data(leads_df)


After removing rows - Shape: (60063, 10)



In [7]:
offers_df = data_cleaning.delete_row_with_duplicate_data(offers_df)


After removing rows - Shape: (6076, 9)

